## Part 1: Carroll & Ostlie problem 29.9

---

Consider a _one-component universe of pressureless dust_.

**(a)** Show that:
$$\Omega(t) = \frac{\rho(t)}{\rho_c(t)} = 1 + \frac{kc^2}{(dR/dt)^2} \tag{29.194}$$
which describes how $\Omega$ varies with time. What does this have to say about the nature of the early universe?


### Sol'n

A few different ways to approach this problem. But C&O have done a lot of the hard work for us, already!
We know
$$ \Omega(t) = \frac{\rho(t)}{\rho_c(t)} = \frac{8 \pi G \rho(t)}{3 H^2(t)} \tag{29.18}$$
and
$$ \left[ \left( \frac{1}{R} \frac{dR}{dt} \right)^2 - \frac{8}{3} \pi G \rho \right] R^2 = -k c^2. \tag{29.10}$$

Rearrange to get
$$ 8 \pi G \rho(t) = 3\left( H^2(t) + \frac{kc^2}{R^2} \right) $$
and substituting this in (29.18) gives:
$$ \Omega(t) = \frac{3 \left( H^2(t) + \frac{kc^2}{R^2} \right)}{3 H^2(t)} = 1 + \frac{kc^2}{H^2(t) R^2}.$$

Now we can just use the Hubble parameter's definition, $H = (dR/dt)/R$, to get:
$$ \Omega(t) = 1 + \frac{kc^2}{(dR/dt)^2}.$$

**What does this say about the early universe?**

$dR/dt \rightarrow \infty$ as $t \rightarrow 0$, so the second term above becomes negligible and $\Omega \approx 1$ at early times.

---

**(b)** Show that $dR/dt \rightarrow \infty$ as $t \rightarrow 0$. What does this say about the difference between a closed, a flat, and an open universe at very early times?

### Sol'n

We can use (29.10) from C&O again:
$$ \left[ \left( \frac{1}{R} \frac{dR}{dt} \right)^2 - \frac{8}{3} \pi G \rho \right] R^2 = -k c^2. \tag{29.10}$$
$$  \left( \frac{dR}{dt} \right)^2 - \frac{8 \pi G \rho_0}{3R} = -k c^2. \tag{29.11}$$

Now, the RHS is constant.
But the second term is proportional to $1/R$ and must diverge as $t \rightarrow 0$ (since $R \rightarrow 0$).
So for very early times, $dR/dt$ must also diverge to $\infty$.

**What does this say about the difference between a closed, a flat, and an open universe at very early times?**

Since $\Omega \rightarrow 1$ (i.e., $\rho \rightarrow \rho_c$) as $t \rightarrow 0$, closed, flat, and open universes become indistinguishable at very early times!

Corollary: small deviations from flatness ("perturbations in curvature") grow over time!

---

## Part 2: Simulating (matter-only) universes

Let's start by simplifying (29.11) a bit.
Define:
$$\dot{R} = \frac{dR}{d(t/t_H)} = H_0^{-1} \frac{dR}{dt},$$
the scale factor's derivative with respect to time, in units of the Hubble time.
Then we have (taking $R_0 = 1$):
$$ \dot{R}^2 = \frac{\Omega_0}{R} + (1 - \Omega_0); $$
$$ \dot{R} = \pm \sqrt{\frac{\Omega_0}{R} + (1 - \Omega_0)}. \tag{1}$$

Finding an analytic relationship between $R$ and $t$ is surprisingly tough, when $\Omega \neq 1$ (i.e., $k \neq 0$).
C&O don't bother to even show how to the integral is done, they just give the result in a mysterious parametric form.

Let's try to recover the same results as C&O (e.g., Fig 29.5) by numerically simulating a single-component pressureless dust universe.
I.e., numerically solving the system of differential equations provided by the Friedmann Equations.

Knowing how to do this will be very useful, when we start adding other components to our universe!

Can we unambiguously solve for $R(t)$ using equation (1) above?

We don't know the sign of $\dot R$! So let's add another equation to our system.
The acceleration equation:
$$ \frac{d^2R}{dt^2} = - \frac{4}{3} \pi G \rho_m R  = -\frac{H_0^2 \Omega_0}{2 R^2}$$ 
$$ \ddot{R} = -\frac{\Omega_0}{2 R^2} \tag{2}$$ 

Now we're ready to solve our system of ODEs.
We could write our own solver, but `scipy` already implements a good one in the `scipy.integrate` module! (Check out the [docs](https://docs.scipy.org/doc/scipy/tutorial/integrate.html#ordinary-differential-equations-solve-ivp)!)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate

`scipy`'s `integrate.solve_ivp` routine will be helpful for us, but only works with first-order (vector) differential equations.
So we a slight change of variables to make our system of equations first-order.

Let
$$ \mathbf{y} = \begin{bmatrix} R \\ \dot{R} \end{bmatrix} $$
then we have
$$ \frac{d\mathbf{y}}{dt} = \frac{d}{dt} \begin{bmatrix} R \\ \dot{R} \end{bmatrix} = \begin{bmatrix} \dot{R} \\ \ddot{R} \end{bmatrix} = \begin{bmatrix} \dot{R} \\ -\frac{\Omega_0}{2R^2} \end{bmatrix},$$
a first-order system in $t$!

In [ ]:
# Let's implement our expressions for \dot{R} and \ddot{R} in code, with numpy's help...

def Rdot(R, Ω0):
    # Eq. (1): derivative of scale factor w.r.t. time (in units of the Hubble time)
    return np.sqrt(Ω0/R + (1 - Ω0))
    
def Rddot(R, Ω0):
    # Eq. (2): second derivative of scale factor w.r.t. time (in units of the Hubble time)
    return -Ω0 / (2 * R**2)

def dydt(t, y, Ω0):
    # We define y = [R, Rdot]
    # So dy/dt = [Rdot, Rddot]
    return np.asarray([y[1], Rddot(y[0], Ω0)])

`solve_ivp` can solve the system of ODEs, starting from an initial value/state ($\mathbf{y_0}$).

What's a good starting point in this case?

In [ ]:
def scale_factor(Ω0=1, tmin=-1, tmax=6, t0=0, max_step=0.1):
    """Solve for scale factor, as a function of time.
    
    Parameters
    ----------
    Ω0 : float
        Matter density at time t = t0.
    tmin, tmax : float
        Solver will stop when it reaches tmin or tmax (in units of the Hubble time)
    t0 : float
        Initial time, in units of the Hubble time. We assume R(t0) = 1
    max_step: float
        Maximum step size, in units of the Hubble time.
        
    Returns
    -------
    t : array[float]
        Times corresponding to each step of the solver, in units of the Hubble time.
    R : array[float]
        Scale factor at each time step.
    """
    
    # initial (t = t0) state; R(t0) = R0 = 1
    R0 = 1
    y0 = [R0, Rdot(R0, Ω0)]
    
    # we solve the system of ODEs twice:
    
    # first, going back in time...
    history = integrate.solve_ivp(dydt, (t0, tmin), y0, args=(Ω0,), max_step=max_step)
    
    # ...and then again, going forward in time
    future = integrate.solve_ivp(dydt, (t0, tmax), y0, args=(Ω0,), max_step=max_step)
    
    # concatenate the results
    # (reorder the history values first, so t is ascending)
    t = np.append(history.t[::-1], future.t)
    R = np.append(history.y[0, ::-1], future.y[0])
    
    return t, R

In [ ]:
# cf. C&O Figure 29.5

fig, ax = plt.subplots(figsize=(6, 5))

ax.plot(*scale_factor(Ω0=0), label='empty')
ax.plot(*scale_factor(Ω0=0.5), label='open')
ax.plot(*scale_factor(Ω0=1), label='flat')
ax.plot(*scale_factor(Ω0=2), label='closed')

ax.legend()

ax.set_xlabel(r'$(\Delta t)/t_H$ from present')
ax.set_ylabel(r'$R$')

fig.tight_layout()

---

## Bonus: Simulating multi-component universes